# 7. 배터리 잔량 확인하기
## ads1115 - 현재 pinky의 배터리 출력관련 모듈

![](../../picture/pinky_blue/part2/02_battery.ipynb_2025-04-22-20-27-33.png)

## 우리는 여기서 I2C 통신을 사용할 예정
- I2C 통신은 2개의 선을 이용
- Master 1개와 여러 개의 Slave가 연결되어 통신이 가능
- 클럭 신호를 사용하는 동기식 통신 방식이기 때문에 시간에 자유로움
- Slave 선택을 위해 항상 주소 데이터가 붙기 때문에 긴 데이터를 전송 하기에는 부적합
![](../../picture/pinky_blue/part2/02_battery.ipynb_2025-04-22-20-28-27.png)

## pinky 폴더 내에 battery.ipynb 파일 생성

![](../../picture/pinky_blue/part2/02_battery.ipynb_2025-04-22-20-30-04.png)

## 필요한 모듈 import

In [1]:
import time  # 시간 모듈
import board  # 보드 정의 모듈
import busio  # I2C 통신 모듈
import adafruit_ads1x15.ads1115 as ADS  # ADS1115 관련 모듈
from adafruit_ads1x15.analog_in import AnalogIn  # 아날로그 입력 모듈

## I2C 버스, ADC, 채널 생성
- I2C 버스: 여러 기기가 하나의 버스로 연결되어 정보를 주고받는 것을 의미함. I2C 버스에서는 두 개의 선 (SDA와 SCL)으로 여러 기기가 연결되어 서로 데이터를 교환할 수 있음
- ADS: 아날로그 신호를 디지털 신호로 변환하는 장치
- 단일 입력: 한 채널을 통해 측정하는 방식으로 전압을 측정할 때 기준 전압(보통 그라운드)과 비교하여 측정

In [4]:
# I2C 버스 생성
i2c = busio.I2C(board.SCL, board.SDA)

# I2C 버스를 사용하는 ADC 객체 생성
ads = ADS.ADS1115(i2c)

# 채널 0에서 단일 입력 생성
chan = AnalogIn(ads, ADS.P0)

## 전압 백분율 계산 함수 작성

In [5]:
def calculate_percentage(value):
    MIN_VALUE = 2396  # 최소 값 정의
    MAX_VALUE = 3215  # 최대 값 정의

    if value < MIN_VALUE:  # 값이 최소값보다 작을 경우
        return 0.0  # 백분율 0 반환
    elif value > MAX_VALUE:  # 값이 최대값보다 클 경우
        return 100.0  # 백분율 100 반환
    else:  # 값이 범위 내에 있을 경우
        percentage = (value - MIN_VALUE) / (MAX_VALUE - MIN_VALUE) * 100  # 백분율 계산
        return percentage  # 계산된 백분율 반환

## 배터리 전압 측정 후 배터리 상태 백분율로 반환하는 함수 작성

In [6]:
def battery():
    i2c = busio.I2C(board.SCL, board.SDA)  # I2C 버스 생성
    ads = ADS.ADS1115(i2c)  # ADC 객체 생성
    chan = AnalogIn(ads, ADS.P0)  # 아날로그 입력 채널 설정

    values = []  # 값을 저장할 리스트 생성

    for _ in range(10):  # 10번 반복
        values.append(chan.value)  # 현재 아날로그 값 추가
        time.sleep(0.1)  # 0.1초 대기

    avg_value = sum(values) / len(values)  # 평균 값 계산
    percentage = calculate_percentage(avg_value)  # 평균값으로 백분율 계산

    return percentage  # 백분율 반환

## 배터리 잔량 출력
- 잔량 출력 시에는 충전기를 빼고 측정하기

In [10]:
print("battery : ", str(battery()), "%")

battery :  97.72893772893774 %
